In [1]:
from supabase import create_client, Client
import os
import requests
import json
import time
model_name = "text-embedding-3-small"
openai_api_key = os.environ['OPENAI_API_KEY']
supabase_url = os.environ['SUPABASE_URL']
supabase_api_key = os.environ['SUPABASE_API_KEY'] 
supabase: Client = create_client(supabase_url, supabase_api_key)

### 請注意，目前的supabase預設只會回傳1000筆資料，若要一口氣拿到所有資料可以考慮
1. 用我下面這塊程式碼的方法
2. https://github.com/orgs/supabase/discussions/3765

In [2]:
page_size = 1000
offset = 0
all_data = []

while True:
    response = supabase.table('drcd_questions').select("embedding, paragraph_id").range(offset, offset + page_size - 1).execute()
    if not response.data:
        break
    all_data.extend(response.data)
    offset += page_size
question_embeddings = [ eval(x["embedding"]) for x in all_data]
gold_paragraph_ids = [x["paragraph_id"] for x in all_data]

print(len(all_data))

3493


In [3]:
response2 = supabase.table('drcd_paragraphs').select("id, embedding").eq("model", model_name).execute()
paragraph_ids = [x["id"] for x in response2.data]
paragraph_embeddings = [ eval(x["embedding"]) for x in response2.data]

print("Dimension:")
print( len(question_embeddings[0]) ) # 這是向量維度

Dimension:
1536


In [4]:
def find_index(arr, target):
  try:
      index = arr.index(target)
      return index
  except ValueError:
      return "not_found"

def calculate_average(arr):
    if len(arr) == 0:
        return 0  # 防止除以零錯誤
    return sum(arr) / len(arr)

def get_embeddings(input, model):
  payload = { "input": input, "model": model }
  headers = { "Authorization": f'Bearer {openai_api_key}', "Content-Type": "application/json" }
  response = requests.post('https://api.openai.com/v1/embeddings', headers = headers, data = json.dumps(payload) )
  obj = json.loads(response.text)
  if response.status_code == 200 :
    return obj["data"][0]["embedding"]
  else:
    time.sleep(3)
    print("embedding error..... retrying")
    # retry
    return get_embeddings(input, model)


### 方法一：用numpy的cosine similarty尋找最相似的內容並輸出結果。

In [10]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# 參數 list_of_doc_vectors 是所有文件的 embeddings 向量
# 參數 query_vector 是查詢字串的 embedding 向量
# 參數 top_k 是回傳的比數
def cosine_similarity_search(list_of_doc_vectors, query_vector, top_k):
  # 轉成 numpy arrays
  list_of_doc_vectors = np.array(list_of_doc_vectors)
  query_vector = np.array(query_vector)

  # 逐筆計算 cosine similarities
  similarities = cosine_similarity(query_vector.reshape(1, -1), list_of_doc_vectors).flatten()

  # 根據 cosine similarity 排序
  sorted_indices = np.argsort(similarities)[::-1]

  # 取出 top K 的索引編號
  top_k_indices = sorted_indices[:top_k]

  return top_k_indices

In [11]:
hit_data = []
mmr_data = []
for idx, question_embedding in enumerate(question_embeddings):

  if idx%10 ==9:
    print(idx)
    break

  best_indexes = cosine_similarity_search(paragraph_embeddings, question_embedding, 5) # 取出 top_k 的 indexes
  print(best_indexes)
  context_ids = [paragraph_ids[i] for i in best_indexes] # 找出對應的 paragraph_ids
  hit_paragraph_id = gold_paragraph_ids[idx] # 這是黃金 paragraph_id

  position = find_index(context_ids, hit_paragraph_id)
  if position == "not_found":
    score = 0
  else:
    score = 1 / (position+1)

  mmr_data.append(score)
  hit_data.append(hit_paragraph_id in context_ids)

average_hit = sum(hit_data) / len(hit_data)

print("---------------------------")
print(average_hit)

average_mrr = calculate_average(mmr_data)

print("MRR score:")
print(average_mrr)

[  8   9 411 384 913]
[  9   8 901 914 384]
[ 10 970  11 500 568]
[11 10 12 16 14]
[14 12 16 11 15]
[ 15  12  16  14 563]
[ 15  12  16  14 508]
[ 17  77 612  25 820]
[ 26  17  25 853  18]
9
---------------------------
1.0
MRR score:
0.9444444444444444


- average score: 0.8685943315201832
- MRR score: 0.7540843591945797
- 時間：約5分鐘

### 方法2:使用langchain的vector search(基底也還是cosine similarity)

我希望可以輸入問題的時候，去尋找drcd_paragraphs裡面最相關的top_k筆資訊

以下是修改後的 SQL 函數，用於在 drcd_paragraphs 表格中搜索最相關的內容：
```sql
create function match_drcd_paragraphs (
  query_embedding vector(1536),
  top_k integer default 5
) returns table (
  content text,
  similarity float
) language plpgsql as $$
begin
  return query
  select 
    content,
    1 - (embedding <=> query_embedding) as similarity
  from drcd_paragraphs
  order by embedding <=> query_embedding
  limit top_k;
end;
$$;
```

In [5]:
from langchain_community.vectorstores import SupabaseVectorStore

def langchain_vector_search_singleask(query, top_k=5):
    # 獲取查詢的 embedding
    query_embedding = get_embeddings(query, model_name)
    
    # 執行相似度搜索
    response = supabase.rpc('match_drcd_paragraphs', {
        'query_embedding': query_embedding,
        'top_k': top_k
    }).execute()
    
    return response.data

In [6]:
import pandas as pd

query = "中國朝代有哪些？"
response = langchain_vector_search_singleask(query, top_k=5)

# 構建一個清理好的 DataFrame
result_data = []
for i in response:
    result_data.append({
        'id': i.get('id', None),  # 如果 'id' 缺失，填 None
        'content': i.get('content', None),
        'similarity': i.get('similarity', None)
    })
result_pd = pd.DataFrame(result_data)
result_pd

,id,content,similarity
0,117,東方由於各個文明發端較早、文明發展呈連續性，各地保留有相當多、種類各異的貴族等級制度相關記載...,0.502288
1,965,明清兩朝因有檔案留存，對於皇帝妃嬪人數有較為全面的了解。明朝嘉靖帝和清朝康熙帝分別是明清兩朝...,0.493501
2,964,在中國歷代的史書中，由於妃嬪人數眾多，相較於皇后，通常對她們很難全部記錄。一位皇帝即使擁有眾...,0.491509
3,905,元朝在推行漢人的典章制度與維護蒙古舊法之間，時常發生衝突，並且分裂成守舊派與崇漢派。早在元太...,0.460949
4,786,朱棣稱帝後，六月十八，恢復了周王朱橚、齊王朱榑的爵位。隨後恢復代王朱桂、岷王朱楩的爵位。永樂...,0.459180


In [14]:
#正式進入評測環節
hit_data = []
mmr_data = []

for idx, question_embedding in enumerate(question_embeddings):
  if idx%5 == 4:
    print(idx)
    break
  # 執行相似度搜索
  response = supabase.rpc('match_drcd_paragraphs', {
      'query_embedding': question_embedding,
      'top_k': 5
  }).execute()
  best_indexes = []
  print(response.data)
  for response in response.data:
    best_indexes.append(response["id"])
  context_ids = [paragraph_ids[i]-1 for i in best_indexes] # 找出對應的 paragraph_ids
  print(context_ids)
  hit_paragraph_id = gold_paragraph_ids[idx] # 這是黃金 paragraph_id
  print(hit_paragraph_id)
  position = find_index(context_ids, hit_paragraph_id)
  if position == "not_found":
    score = 0
  else:
    score = 1 / (position+1)

  mmr_data.append(score)
  hit_data.append(hit_paragraph_id in context_ids)

average_hit = sum(hit_data) / len(hit_data)

print("---------------------------")
print(average_hit)

average_mrr = calculate_average(mmr_data)

print("MRR score:")
print(average_mrr)


[{'id': 9, 'content': '儒略日是在儒略週期內以連續的日數計算時間的計時法，主要是天文學家在使用。儒略日數的計算是從格林威治標準時間的中午開始，包含一個整天的時間，起點的時間回溯至儒略曆的西元前4713年1月1日中午12點，這個日期是三種多年週期的共同起點，且是歷史上最接近現代的一個起點。例如，2000年1月1日的世界協調時間12:00是儒略日2,451,545。儒略日期是以格林威治標準時中午12:00的儒略日加上那一天的瞬時時間的分數。儒略日期是儒略日添加小數部分所表示的儒略日數。例如，2013年1月1日00:30:00是儒略日期2,456,293.520833。', 'similarity': 0.698032140731831}, {'id': 10, 'content': '儒略週期是開始於西元前4713年，長達7980年的紀年法，被用於歷史上各種不同曆法的日期轉換。西元2017年是儒略週期的6730年，下一個儒略週期將開始於西元3268年。儒略日的起點訂在西元前4713年1月1日格林威治時間平午，即儒略日0指定為世界協調時間西元前4713年1月1日12:00到世界協調時間西元前4713年1月2日12:00的24小時。每一天賦予了一個唯一的數字，順數而下，如：1996年1月1日12:00:00的儒略日是2450084。這個日期是考慮了太陽、月亮的軌道運行周期，以及當時收稅的間隔而訂出來的。约瑟夫·斯卡利格定義儒略周期為7980年，是因28、19、15的最小公倍數為28×19×15=7980。', 'similarity': 0.62354506149338}, {'id': 399, 'content': '《聖經》在《詩篇》93:1, 96:10，以及《歷代志》上16:30記錄著世界就堅定，不得動搖。在《詩篇》104:5以同樣的方式寫著將地立在根基上，使地永不動搖。更多地，在《傳道書》1:5中記錄著日頭出來，日頭落下，急歸所出之地。等等。和焦爾達諾·布魯諾一樣，伽利略捍衛日心說，進而宣稱它與經文並不衝突。他採取奧古斯丁的經文立場，即不能將所有篇章予以字面解釋，特別是在詩篇和歌頌上，不是對現實的描述或歷史問題上。他相信經文作者僅僅是從地理角度來觀察世界，從這點來看太陽的確升起落下。另一種解釋是作者從一種現象的角度來記錄，或是這只是一種寫作

In [ ]:
[  8   9 411 384 913]
[  9   8 901 914 384]
[ 10 970  11 500 568]
